In [ ]:
. ${KALDI_INSTRUCTIONAL_PATH}/path.sh

# 6.4: Examining the acoustic models

Unfortunately, there isn't an easy way to evaluate the quality of an acoustic model in isolation.  The best we will try to do in this notebook is gather some intuition on two aspects of our model:
 1. the relationship between the number of phones we need to model and the size/shape of our `decision` `tree`
 2. the impact of the two hyperparameters, `num_leaves`, and `num_gaussians`.

We will do this by looking at the two primary files generated at each layer of the acoustic training: `tree` and `*.mdl`.

## understanding the `tree` file

The `tree` file is a `binary` representation of the decision tree built during the acoustic training.  This `tree` contains information about which phones were clustered together (*e.g.* "state-tied") to reduce the space of possibilities we need to model.

In the end, each `leaf` of the tree will represent a probability distribution (`pdf`), and the `tree`'s job is to decide which phones can be grouped together appropriately.

### `tree-info`

We can use `tree-info` to get a few useful pieces of information about the `decision` tree.

In [ ]:
tree-info

In [ ]:
tree-info exp/monophones/tree

`num-pdfs` is the number of distributions that we end up with in the `tree`.  Remember, these are the `leaves`.  

`context-width` refers to how many phones of "context" we included.  `central-position` refers to which item is the "central" phone (and which are the "context" phones).  Since the tree above was built from `monophones` the width was `1` and the `central-position` was `0`.  But compare to the `tree` from the `triphones`.

In [ ]:
tree-info exp/triphones/tree

Now we have a `width` of 3, meaning one phone of "context" on each side, and thus the `central-position` is the second phone (remember, we're using `0-indexing` here).

#### `num_leaves` in relation to `num-pdfs`

If you recall, one of our hyperparameters was called `num_leaves`.  This was the approximate number of leaves we wanted to allow our `tree` to generate.  

In general, there is a trade-off between generalization and over-fitting in decision trees.  If you end up with a tree that has the same number of leaves as number of data points, then you achieved no generalization.  In the opposite case, if you end up with a tree that only has one leaf, then you maximized your generalization.  Neither case is ideal.  

**Note**: We'll capture this relationship in a ratio later in the notebook.

In our case, one thing we *can* easily inspect is how well our `tree` did in clustering similar phones together.

Each phone is represented by a 3-state `HMM` (techinally, `SIL` is often modeled by a 5-state `HMM`, but we'll assume 3-states for the sake of this discussion).  And so in the case of `monophone` training, that means we have $num\_phones * 3$ different states to model.  

In [ ]:
cat raw_data/librispeech-phones.txt | wc -l

Our setup has 70 phones (this is a lot but it's because the `Librispeech` datset chose to model different forms of the same vowel).  And so our `monophone` model needs to handle 210 total conditions.

##### `monophone` model

In [ ]:
tree-info exp/monophones/tree | grep num-pdfs

You'll notice the `monophone` `tree` ended up with 212 leaves.  This is because `SIL` was modeled by 5-states instead of 3.

This also means that **no** clustering was done.  In other words, we have the same number of leaves as possible states.  The `tree` is used here **only** to be consistent with the other layers of acoustic training, but it really doesn't do anything.

This can be verified by looking more closely at the `monophone` tree itself.  There are close-ups of the output from `draw-tree` that we ran in the previous notebook in `resource_files/tree_viz`.  They are rendered below.

![tree_out](resource_files/tree_viz/monophone_tree_zoom_out.png)

![tree_in](resource_files/tree_viz/monophone_tree_zoom_in.png)

You'll notice that each split of the tree is simply splitting by phone.  In the closeup above, you'll see that the top split shown sends all the `SIL` phones to the left and all the `AA1` phones to the right.

The final split (before reaching the leaves) decides which state of the `HMM` we are in.  And here you'll see confirmation that the `SIL` phone is modeled by a `5-state` `HMM` while the others are `3-state` `HMM`s.

We can also get a `text`-version of any `tree` by using `copy-tree` and the `--binary=false` flag.

In [ ]:
copy-tree

In [ ]:
copy-tree --binary=false exp/triphones/tree - | head

A line that starts with `SE` (which stands for `SplitEventMap`) indicates a place where we are splitting the branches of the tree.  And a line with `CE` (which stands for `ConstantEventMap`) means we have hit a terminal node (or a `leaf`).  So the number of `CE`s will equal `num-pdfs` that we saw from `tree-info`,and the number of `SE`s will equal the total number of splits we made.

In the visualization below, each `oval` is an `SE` and each `double-circle` is a `CE`.

![tree_in](resource_files/tree_viz/monophone_tree_zoom_in.png)

In [ ]:
copy-tree --binary=false exp/monophones/tree - | grep SE | wc -l

Our `monophone` tree only made 69 total splits.  In other words, it only needed `69` splits to handle the `70` total possible phones. Not surprising since, in the end, our `monophone` tree did no "real" clustering, and just deposited each `phone` on its own `leaf`.  This is essentially a `1-to-1` ratio of `number_of_phones-to-number_of_splits`.

We can calculate this ratio by capturing two values:
 - `num_phones` (just counting the number of lines in our `phones.txt` file) 
 - `num_splits` (counting the number of `SE`s in the `text` version of the `tree`)

In [ ]:
num_phones_mono=$(cat raw_data/librispeech-phones.txt | wc -l)
num_splits_mono=$(( $(copy-tree --binary=false exp/monophones/tree - | grep SE | wc -l) ))  

Then we'll just divide the two values.

In [ ]:
echo print ${num_phones_mono}/${num_splits_mono}. | perl

##### `triphone` model

Now we should expect two things to be true about our `triphone` tree.
 1. more `pdf`s (*i.e.* `leaves`) than the `monophone` tree because we are now modeling "context" which means `num_phones^3` possible phones (in our case, `70*70*70` possible combinations)
 2. less than `70*70*70` total `pdf`s, though, since we are expecting our `tree` to have succeeded in some generalization.

In [ ]:
tree-info exp/triphones/tree | grep num-pdfs

Since we started off with `70` possible phones, we could have up to `343,000` total combinations.  Above you can see that our `triphones` `tree` reduced that significantly, and so we assume this means "better" clustering going on.  How "good" the clustering is can be approximated by that `num_phones-to-num_splits` ratio we calculated earlier.  The *larger* that number is the "better" the clustering we were able to accomplish (because we were able to "handle" *more* phones in a single split).

This time, though, we can't just count the number of lines in our `phones.txt` file, we have to take that number to the third power.

In [ ]:
num_phones_in_context=$(( ${num_phones_mono} * ${num_phones_mono} * ${num_phones_mono} ))
num_splits_tri=$(( $(copy-tree --binary=false exp/triphones/tree - | grep SE | wc -l) ))  

In [ ]:
echo print ${num_phones_in_context}/${num_splits_tri}. | perl

This means that we were able to, on average, cluster about `90` phones in context into one `leaf` of our tree. 

**Note:** We discussed before the "fine line" we try to balance with `decision tree`s.  We want to generalize, but not too much.  And we want to cluster, but not too much.  So let's not make the mistake of thinking that the goal is to get the largest possible `num_phones-to-num_splits` ratio possible.

We *do* have *some* control over this, however, with the `num_leaves` hyperparameter.  This value tells `kaldi` the approximate maximum number of `leaves` we wanted in our tree.  For the `triphone` `tree` and in our code, that was set to `num_leaves=5000`.

In [ ]:
cat exp/triphones/kaldi_config_args.json

In [ ]:
tree-info exp/triphones/tree | grep num-pdfs

It's not easy to figure out how `kaldi` decides exactly how many `leaves` it needs.  But it turns out the `triphone` `tree` didn't require `5000 leaves` at all.

##### `triphone_lda` model

One of the things that the `run_train_phones.sh` script does is *double* the number of `leaves` for the `LDA` model.  So even though our `config` says `num_leaves=5000`, the code will make that `10000`

In [ ]:
cat exp/triphones_lda/kaldi_config_args.json

In [ ]:
tree-info exp/triphones_lda/tree | grep pdf

And so it's shouldn't be surprising that our `lda` `tree` ended up with more `leaves` than the `triphone` `tree`.  And since the number of `phones` in context didn't change (still `70*70*70`), this tree should have a *smaller* `num_leaves-to-num_splits` ratio.

In [ ]:
num_splits_lda=$(( $(copy-tree --binary=false exp/triphones_lda/tree - | grep SE | wc -l) ))  

In [ ]:
echo print ${num_phones_in_context}/${num_splits_lda}. | perl

In this case, a *smaller* ratio means that were allowed to make *smaller* clusters (fewer phones in the same `leaf`) in this `tree`.  The hope is that translates to more accurate clustering (it's more "specific" than the `triphone` `tree`'s clustering).  Unfortunately, there's no easy way, at this point in our ASR pipeline, to test that assumption.  

## understanding the `.mdl` file

The `.mdl` file is the final `Hidden Markov Model` (`HMM`) representing the acoustic model.  But it also gives us some information about the `Gaussian Mixture Model`s (`GMM`s) we built to represent the different phones.

### `gmm-info`

`gmm-info` will give use some very general statistics about the model, including some information about the `GMM` and some information about the `HMM`.  

In [ ]:
gmm-info

In [ ]:
gmm-info exp/monophones/final.mdl 

#### `num_leaves` in relation to `num_gaussians`

We already looked at the relationship between `num_phones` and `num-pdfs`, so now we'll look at the relationship between two hyperparameters:
 - `num_leaves` (which ends up being represented as `num-pdfs` in the `tree`) 
 - `num_gaussians`

Each `pdf` is a probability distribution that looks like some version of a bell-curve (*i.e.* a `Gaussian` distribution).  A `GMM` is a "mixture-model" meaning, we combine *multiple* `Gaussian` distributions to get a "more sophisticated" representation.  But the same warning arises here as we had for `decision tree`s.  We want to find the "sweet spot" between generalization (*i.e.* *one* `Gaussian` distribution) and over-fitting (*i.e.* *too many* `Guassian` distributions).

And so we have another hyperparameter, called `num_gaussians` that we set.

In [ ]:
cat exp/monophones/kaldi_config_args.json

In general, we can create a ratio (similar to what we did for our `decision` `tree`) of, on average, how many `Gaussian`s we used for each `pdf`.

We simply need to divide the `number of gaussians` by the `number of pdfs`.

#### `monophone` tree

In [ ]:
num_gaussians_mono=$(gmm-info exp/monophones/final.mdl | grep -Po "(?<=number of gaussians )[0-9]+")
num_pdfs_mono=$(gmm-info exp/monophones/final.mdl | grep -Po "(?<=number of pdfs )[0-9]+")

echo ""
echo "number of gaussians: ${num_gaussians_mono}"
echo "number of pdfs: ${num_pdfs_mono}"

In [ ]:
echo print ${num_gaussians_mono}/${num_pdfs_mono}. | perl

This relatively high ratio value, means that, on average, we were able to use about `45` `Gaussian`s to model each `phone`.

#### `triphone` tree

In [ ]:
num_gaussians_tri=$(gmm-info exp/triphones/final.mdl | grep -Po "(?<=number of gaussians )[0-9]+")
num_pdfs_tri=$(gmm-info exp/triphones/final.mdl | grep -Po "(?<=number of pdfs )[0-9]+")

echo ""
echo "number of gaussians: ${num_gaussians_tri}"
echo "number of pdfs: ${num_pdfs_tri}"

In [ ]:
echo print ${num_gaussians_tri}/${num_pdfs_tri}. | perl

This ratio drops significantly when we have to model phones in context, and we're still given the same number of `Gaussian`s.

#### `triphone_lda` tree

Just like the `run_train_phones.sh` code doubled the `num_leaves` for the `triphone_lda` `tree`, it also doubled the `num_gaussians` for the `triphone_lda` `mdl`.

In [ ]:
num_gaussians_lda=$(gmm-info exp/triphones_lda/final.mdl | grep -Po "(?<=number of gaussians )[0-9]+")
num_pdfs_lda=$(gmm-info exp/triphones_lda/final.mdl | grep -Po "(?<=number of pdfs )[0-9]+")

echo ""
echo "number of gaussians: ${num_gaussians_lda}"
echo "number of pdfs: ${num_pdfs_lda}"

In [ ]:
echo print ${num_gaussians_lda}/${num_pdfs_lda}. | perl

And so even though the number of `pdfs` we had increased from the `triphones` `tree`, because we were given twice as many `Gaussian`s, we were able to increase the ratio value for the `triphon_lda` `mdl`.

## "best" settings for these hyperparameters

Again, since we have no easy way of assessing the quality of our acoustic models at this point in the ASR pipeline, we are unable to empirically find the "sweet spot" for this ration.

In the case of setting these two specific hyperparameters (`num_leaves` and `num_gaussians`), the only option we have would be to look at the downstream metric of `Word Error Rate` (`WER`) of the whole system to get an idea of what the perfect ratio is.

But we *can* come up with a general intuition:
 - setting `num_leaves` higher will allow the `decision` `tree` to be *more specific* in how it clusters phones (*e.g.* will cluster fewer phones together); setting it `lower` will force the `tree` to generalize more
   - minimally, we 
 - setting `num_gaussians` higher will allow the `GMM` to be *more specific* in how many `Gaussian`s it uses to model each `pdf`; setting it `lower` will force the `GMM` to generalize more
 - `num_gaussians` divided by `num_leaves` will indicate, on average, how many `Gaussian`s you want to contribute to each `pdf`